In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

from scripts.visualizer import Visualizer

In [30]:
### Preparing for visualization
# Config
weekdays=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

path_aktualne = ("aktualne", "data/articles_Aktualne.cz.csv")
path_idnes = ("idnes", "data/articles_iDnes.cz.csv")
file_paths = [path_aktualne, path_idnes]

v = Visualizer(files=file_paths, verbose=True)

  0%|                                                    | 0/2 [00:00<?, ?it/s]

Data from data/articles_Aktualne.cz.csv successfully uploaded.


 50%|██████████████████████                      | 1/2 [00:00<00:00,  7.88it/s]

Data from data/articles_iDnes.cz.csv successfully uploaded.


100%|████████████████████████████████████████████| 2/2 [00:00<00:00,  7.05it/s]


Visualizer successfully initialized.


In [31]:
def getDay(data):
    '''
        Creates new column with weekday from the date
    '''
    
    data = data.dropna().copy()
    data["date"] = pd.to_datetime(data["date"])
    data["weekday"] = pd.Categorical(data["date"].dt.day_name(), categories = weekdays, ordered = True)
    

    return data

data_days = getDay(v.data)
data_days

,article_id,slug,date,time,is_updated,headline,excerpt,article_url,scraped_at,source,weekday
1,r~490a15268b0911ea8972ac1f6b220ee8,celou-vec-jsem-otevrene-vysvetlil-rekl-mynar-p...,2020-04-30,20:01:00,True,"Z Hradu na Kuberu tlak nebyl, celou věc jsem o...","Kancléř prezidenta Vratislav Mynář odmítl, že ...",/domaci//domaci/celou-vec-jsem-otevrene-vysvet...,03-05-2020 19:47:53,aktualne,Thursday
2,r~70418c3c8aff11ea842f0cc47ab5f122,vlada-schvalila-navrh-na-zruseni-dane-z-nabyti...,2020-04-30,19:54:00,True,Vláda schválila návrh na zrušení daně z nabytí...,Zároveň také prodloužila z pěti na deset let t...,/domaci//ekonomika/vlada-schvalila-navrh-na-zr...,03-05-2020 19:47:54,aktualne,Thursday
3,r~73da20908b0711eaa6f6ac1f6b220ee8,pametni-deska-pripominajici-vlasovce-byla-v-re...,2020-04-30,19:43:00,True,V Řeporyjích i přes ruské protesty instalovali...,Plastika zhruba velikosti dlaně zobrazuje sově...,/domaci//domaci/pametni-deska-pripominajici-vl...,03-05-2020 19:47:54,aktualne,Thursday
4,r~9d3006868af211eaa7deac1f6b220ee8,prezident-zeman-telefonicky-mluvil-s-cinskym-p...,2020-04-30,17:59:00,True,Prezident Zeman telefonicky mluvil s čínským p...,Český prezident potvrdil účast na summitu 17+1...,/domaci//domaci/prezident-zeman-telefonicky-ml...,03-05-2020 19:47:54,aktualne,Thursday
5,r~167aef8a8af611eab115ac1f6b220ee8,rousky-nebudou-muset-nosit-deti-ve-skolkach-um...,2020-04-30,17:46:00,True,"Roušky nebudou muset nosit děti ve školkách, u...",Už dříve dostali výjimku řidiči veřejné doprav...,/domaci//domaci/rousky-nebudou-muset-nosit-det...,03-05-2020 19:47:54,aktualne,Thursday
...,...,...,...,...,...,...,...,...,...,...,...
3103,A200428_140551_domaci_remy,graubner-duka-ceska-biskupska-konference-zneuz...,2020-04-28,15:53:00,False,Kardinál Duka končí. V čele biskupů ho po dese...,Novým předsedou České biskupské konference se ...,https://www.idnes.cz/zpravy/domaci/graubner-du...,03-05-2020 22:40:35,idnes,Tuesday
3104,A200428_133040_vary-zpravy_ba,valdstejn-kun-exponat-muzeum-cheb-stehovani-va...,2020-04-28,15:35:00,False,Po lomených schodech v muzeu stěhovalo šest mu...,"Výjimečný kůň Albrechta z Valdštejna, který se...",https://www.idnes.cz/karlovy-vary/zpravy/valds...,03-05-2020 22:40:35,idnes,Tuesday
3105,A200428_100622_domaci_lre,nemocnice-na-homolce-odmeny-pro-sestry-zadne-n...,2020-04-28,14:54:00,False,"Snížíme odměny, napsal překvapeným zdravotníků...","V nemocnicích tráví dlouhé hodiny, zachraňují ...",https://www.idnes.cz/zpravy/domaci/nemocnice-n...,03-05-2020 22:40:35,idnes,Tuesday
3106,A200428_142241_domaci_remy,koronavirus-nouzovy-stav-hrozi-druha-vlna-epid...,2020-04-28,14:28:00,False,"Bez stavu nouze hrozí druhá vlna epidemie, var...",Bez prodloužení nouzového stavu by podle praco...,https://www.idnes.cz/zpravy/domaci/koronavirus...,03-05-2020 22:40:35,idnes,Tuesday


In [32]:
days_count = data_days[["weekday", "source"]].groupby(["weekday", "source"]).size()
days_count = days_count.sort_index()

In [35]:
fig1 = go.Figure(data=[
    go.Bar(name='iDnes.cz', x=weekdays, y=days_count.xs("idnes", level=1)),
    go.Bar(name='Aktualne.cz', x=weekdays, y=days_count.xs("aktualne", level=1))
])
# Change the bar mode
fig1.update_layout(barmode='stack')
fig1.show()

In [25]:
days_count.sort_index()

weekday    source  
Monday     aktualne    247
           idnes       228
Tuesday    aktualne    264
           idnes       246
Wednesday  aktualne    260
           idnes       262
Thursday   aktualne    296
           idnes       279
Friday     aktualne    230
           idnes       245
Saturday   aktualne    106
           idnes       135
Sunday     aktualne    112
           idnes       153
dtype: int64